In [47]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder

os.chdir(r'd:/Murgi/code/memes2024/meme-research-2024')

meme_df = pd.read_parquet('./data/meme_entries.parquet')
labelencoder = LabelEncoder()
meme_df['template_id'] = labelencoder.fit_transform(meme_df['template_name'])
meme_df['template_id'] = meme_df['template_id'] + 1

template_dict = {k: v for k, v in zip(meme_df['template_id'], meme_df['template_name'])}
template_dict[0] = 'templateless'


In [48]:
with open('./sparse_matching_sm_preds.txt', 'r') as f:
    lines = f.readlines()

paths = []
preds = []
for i, line in enumerate(lines):
    try:
        line = line.strip()
        path, pred = line.split('\t')
        paths.append(path)
        preds.append(pred)
    except:
        print(i,line)
df = pd.DataFrame({'path': paths, 'pred': preds})
# cast pred to int32
df['pred'] = df['pred'].astype('int32')
df

,path,pred
0,C:\Users\molontay\Murgi\data\Memes2022Final2\M...,579
1,C:\Users\molontay\Murgi\data\Memes2022Final2\M...,0
2,C:\Users\molontay\Murgi\data\Memes2022Final2\M...,0
3,C:\Users\molontay\Murgi\data\Memes2022Final2\M...,0
4,C:\Users\molontay\Murgi\data\Memes2022Final2\M...,0
...,...,...
108289,C:\Users\molontay\Murgi\data\Memes2022Final2\M...,0
108290,C:\Users\molontay\Murgi\data\Memes2022Final2\M...,0
108291,C:\Users\molontay\Murgi\data\Memes2022Final2\M...,0
108292,C:\Users\molontay\Murgi\data\Memes2022Final2\M...,0


In [53]:
df['predicted_template'] = df['pred'].map(template_dict)

def add_source(row):
    # if the path contains 'train' then it's a training meme
    if 'Memes2022Final2' in row['path']:
        return 'reddit'
    # if the path contains 'val' then it's a validation meme
    elif 'Facebook2023' in row['path']:
        return 'facebook'
    # if the path contains 'test' then it's a test meme
    elif 'Twitter2023' in row['path']:
        return 'twitter'
    
    
df['source'] = df.apply(add_source, axis=1)
df.source.value_counts()

source
reddit      97087
twitter     10207
facebook     1000
Name: count, dtype: int64

In [58]:
df.loc[:,'path'] = df['path'].str.replace('\\', '/')
df['id'] = df['path'].apply(lambda x: os.path.basename(os.path.splitext(x)[0]))
df.to_parquet('./data/sparse_matching/sparse_matching_sm_preds.parquet', index=False)
